In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Prepare cudf
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
#%load_ext cudf.pandas  # pandas operations now use the GPU!
#!pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
#!rm -rf /usr/local/lib/python3.8/dist-packages/cupy*
#!pip install cupy-cuda11x

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 10, in <module>
    for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
KeyboardInterrupt
^C


In [ ]:
# Utilities
#import cudf as pd
import pandas as pd
from tqdm import tqdm

# Text2Vec
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import numpy as np

# Sentiment Analysis
import nltk
# Download the WordNet data (needed for lemmatization)
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download("vader_lexicon")
nltk.download("punkt")
from nltk.sentiment import SentimentIntensityAnalyzer

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sia = SentimentIntensityAnalyzer()

#df = pd.read_csv(drive_path + 'Amazon_Unlocked_Mobile.csv').to_pandas()
df = pd.read_csv(drive_path + 'Amazon_Unlocked_Mobile.csv')
products = set(df["Product Name"])
id_dict = {i: j for j,i in enumerate(products)}
df["ID"] = df["Product Name"].map(id_dict)


df["sentiment"] = [0] * len(df)

for i in tqdm(range(0, len(df) -1)):
    df["sentiment"][i] = sia.polarity_scores(str(df["Reviews"][i]))["compound"]

df = df.drop_duplicates(subset=['ID', 'Reviews'])

df.to_csv(drive_path + "Amazon_Unlocked_Mobile_ID_dedup_sentiment.csv", index=False)
# Sample text

  0%|          | 0/413839 [00:00<?, ?it/s]<ipython-input-23-7e9878bbc308>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"][i] = sia.polarity_scores(str(df["Reviews"][i]))["compound"]
100%|██████████| 413839/413839 [03:47<00:00, 1821.92it/s]


In [ ]:

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
#df = pd.read_csv(drive_path + "Amazon_Unlocked_Mobile_ID_dedup_sentiment.csv")

#df.insert(0, "embeddings", [0] * len(df))
df["cluster"] = [""] * len(df)
df["category"] = [""] * len(df)

categories = ["Technology", "Social", "Price", "Shipping and Packaging", "Customer Service", "General"]
embedded_categories = embed(categories)

for i in tqdm(range(0, max(df["ID"])+1)):
    if len(df["Reviews"].loc[df["ID"] == i]) < 860: #?
        continue
    reviews = df["Reviews"].loc[df["ID"] == i].astype(str).tolist()
    embeddings = embed(reviews)
    df["category"].loc[df["ID"] == i] = [categories[min([(category_id, np.linalg.norm(embedding-category)) for category_id, category in enumerate(embedded_categories)], key=lambda x: x[1])[0]] for i, embedding in enumerate(embeddings)]
    #PCA:
    X = np.vstack(embeddings)
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(X.T)
    pca = PCA(n_components=0.9, svd_solver='full')
    principal_components = pca.fit_transform(X_standardized.T)
    #CLUSTERING:
    silhouette_scores = []
    print("----------------------")
    print(f"i is {i}")
    print(len(df["Reviews"].loc[df["ID"] == i]))
    print(len(principal_components))
    for k in range(int(len(principal_components) / 20), min(len(principal_components) - 1, int(len(principal_components) / 10))):
        kmeans = KMeans(n_clusters=k, random_state=42)
        predicted_labels = kmeans.fit_predict(principal_components)
        silhouette_scores.append(silhouette_score(principal_components, predicted_labels))
    optimal_k = np.argmax(silhouette_scores) + int(len(principal_components) / 20)
    print(optimal_k)
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    predicted_labels = kmeans.fit_predict(principal_components)
    df["cluster"].loc[df["ID"] == i] = [label for label in predicted_labels]
    #df["embeddings"].loc[df["ID"] == i] = [str(embedding.numpy()) for embedding in emmbeddings]

#print(df.loc[df["ID"] == 1])

df = df.dropna(subset=['cluster'], axis=0, how='any')
df.to_csv(drive_path + "Amazon_Unlocked_Mobile_ID_dedup_sentiment_embeddings.csv", index=False)
# Sample text

 37%|███▋      | 1612/4410 [00:00<00:01, 1952.81it/s]<ipython-input-12-35f313cae788>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["category"].loc[df["ID"] == i] = [categories[min([(category_id, np.linalg.norm(embedding-category)) for category_id, category in enumerate(embedded_categories)], key=lambda x: x[1])[0]] for i, embedding in enumerate(embeddings)]


----------------------
i is 1733
868
868


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

63


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-12-35f313cae788>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cluster"].loc[df["ID"] == i] = [label for label in predicted_labels]
 61%|██████▏   | 2712/4410 [01:09<00:30, 55.20it/s]<ipython-input-12-35f313cae788>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["category"].loc[df["ID"] == i] = [categories[min([(category_id, np.linalg.norm(embedding-category)) fo

...

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-12-35f313cae788>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cluster"].loc[df["ID"] == i] = [label for label in predicted_labels]
 79%|███████▊  | 3468/4410 [03:26<02:20,  6.71it/s]<ipython-input-12-35f313cae788>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["category"].loc[df["ID"] == i] = [categories[min([(category_id, np.linalg.norm(embedding-category)) fo

----------------------
i is 3472
877
877


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

75


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-12-35f313cae788>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cluster"].loc[df["ID"] == i] = [label for label in predicted_labels]
100%|██████████| 4410/4410 [04:37<00:00, 15.90it/s]


In [ ]:
sia = SentimentIntensityAnalyzer()

df = pd.read_csv(drive_path + "Amazon_Unlocked_Mobile_ID_embeddings.csv")

df["sentiment"] = [0] * len(df)

for i in tqdm(range(1, len(df))):
    df["sentiment"][i] = sia.polarity_scores(str(df["Reviews"][i]))["compound"]

df.to_csv(drive_path + "Amazon_Unlocked_Mobile_ID_embeddings_sentiment.csv", index=False)

  0%|          | 0/413839 [00:00<?, ?it/s]<ipython-input-3-717ce6954587>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"][i] = sia.polarity_scores(str(df["Reviews"][i]))["compound"]
100%|██████████| 413839/413839 [03:43<00:00, 1849.95it/s]


In [ ]:
df = pd.read_csv(drive_path + "Amazon_Unlocked_Mobile_ID_embeddings_sentiment.csv")

df = df.drop_duplicates(subset=['ID', 'Reviews'])
df = df.dropna(subset=['cluster'], axis=0, how='any')
df.to_csv(drive_path + "Amazon_Unlocked_Mobile_ID_embeddings_sentiment_dedup.csv", index=False)

In [ ]:
print(sorted([(i,len(df.loc[df["ID"] == i])) for i in range(max(df["ID"]))],
    key=lambda x: -x[1]))

[(3472, 877), (2767, 875), (1733, 868), (3467, 862), (2206, 834), (4244, 833), (1232, 831), (1115, 808), (2923, 808), (1562, 800), (995, 775), (1336, 773), (3043, 767), (4021, 767), (1683, 766), (963, 762), (2012, 757), (1199, 756), (253, 745), (763, 742), (1227, 740), (2924, 734), (1926, 729), (2269, 729), (2230, 725), (4175, 716), (1515, 713), (1783, 713), (2480, 713), (3567, 713), (2389, 711), (2155, 710), (677, 709), (2023, 708), (3436, 706), (1439, 699), (486, 695), (3644, 692), (1785, 679), (526, 674), (3835, 672), (3241, 670), (727, 662), (4369, 657), (988, 656), (2768, 656), (423, 655), (472, 655), (4039, 653), (3099, 650), (772, 649), (902, 649), (1555, 649), (2926, 649), (2527, 646), (715, 645), (2166, 645), (3341, 645), (2816, 643), (1406, 642), (3589, 635), (3708, 635), (2893, 629), (3690, 624), (0, 623), (578, 623), (2461, 621), (175, 620), (769, 620), (1826, 620), (2129, 620), (673, 616), (1181, 613), (2028, 607), (3611, 606), (106, 604), (1127, 604), (1670, 604), (3162, 

In [ ]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,ID,sentiment,cluster,category
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,798,0.8783,,
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,798,0.9231,,
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,798,0.4927,,
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,798,0.9185,,
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,798,0.2942,,
...,...,...,...,...,...,...,...,...,...,...
336597,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,"Great phone. Large keys, best flip phone I hav...",0.0,4068,0.8622,,
336598,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,"Pros...Works great, very durable, easy to navi...",0.0,4068,0.5954,,
336599,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,just as described perfect for the price,0.0,4068,0.5719,,
336600,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,Would not work,0.0,4068,0.0000,,


In [ ]:
df = df[df['cluster'] != '']

In [27]:
def count_words_frequency(text):
    # Tokenize the text into words using regular expression
    words = re.findall(r'\b\w+\b', text.lower())  # Convert to lowercase for case-insensitive counting

    # Use Counter to count the frequency of each word
    word_frequency = Counter(words)
    if 'i' in word_frequency:
        del word_frequency['i']
    if len(word_frequency) == 0:
      return ""
    return max(word_frequency, key=word_frequency.get)


def lemmatize_selected_pos(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Perform part-of-speech tagging
    pos_tags = pos_tag(words)

    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Define the set of allowed POS tags (nouns, adjectives, and verbs)
    allowed_pos = {'N'}  # Noun, Verb, Adverb, Adjective

    # Lemmatize words based on the allowed POS tags
    lemmatized_words = [lemmatizer.lemmatize(word, pos=pos[0].lower())
                        for word, pos in pos_tags if pos[0].upper() in allowed_pos]

    # Join the lemmatized words back into a string
    lemmatized_text = ' '.join(lemmatized_words)

    return lemmatized_text


In [28]:
df["Cluster_interpretation"] = [""] * len(df)
for i in range(max(df["ID"])+1):
  j = 0
  while True:
    if (len(df.loc[(df["ID"] == i)&(df["cluster"] == j)]) != 0):
      revs = ' '.join(df.loc[(df["ID"] == i)&(df["cluster"] == j)]["Reviews"].astype(str).tolist()).lower()
      #print(revs)
      #print(count_words_frequency(lemmatize_selected_pos(revs)))
      df.loc[(df["ID"] == i)&(df["cluster"] == j), "Cluster_interpretation"] = [count_words_frequency(lemmatize_selected_pos(revs))]*len(df.loc[(df["ID"] == i)&(df["cluster"] == j)])
      j += 1
      #print(df.loc[df["Cluster_interpretation"] != ""])
    else:
      break

df.to_csv(drive_path + "Amazon_Unlocked_Mobile_ID_dedup_sentiment_embeddings_interpretation.csv", index=False)